Importing the Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable



Importing the datasets

In [3]:
movies=pd.read_csv('ml-1m/movies.dat',sep='::',header=None, engine='python', encoding='latin-1' ) #Movie_ID is noted
users=pd.read_csv('ml-1m/users.dat',sep='::',header=None, engine='python', encoding='latin-1' )#User_ID is noted so that we know what user is using this
ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',header=None, engine='python', encoding='latin-1' )#Rating for the movie against the user is noted

Creating Training and Testing Test

We can see that in many training and testing sets are present/ This is done to perform K-Fold Cross Validation

In [4]:
training_set= pd.read_csv('ml-100k/u1.base', delimiter='\t')
#This should be converted into an array
training_set=np.array(training_set, dtype= 'int')
#The dataset is now an array with training set

In [5]:
test_set= pd.read_csv('ml-100k/u1.test', delimiter='\t')
#This should be converted into an array
test_set=np.array(test_set, dtype= 'int')

We will be separating the two variables - The users and Rating to convert into the matrix. This will create a cell with (U,I) where U is user and I is Movie ID, and the value is the rating of the user for the particular movie.  

Getting thenumber of users and the movies

In [6]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0]))) #total number of users and it is represented as the first coloumn. Maximum no of users is considered/
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1]))) #movie is in the second coloumn and we take that

Next we convert it into the matrix and these coloumns are transferred into the coloumns of the matrix. We need to make a specific data structure, for the restricted boltzmann machine. 

Converting the data into the array with the users in the lines and movies in the coloumns

In [7]:
def convert(data):
  new_data = [] #Final list 
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users] #We take all the movie IDs as the id_users that is in the for loop
    id_ratings = data[:, 2] [data[:, 0] == id_users] #RATINGS of the movie by the user from the second coloumn
    ratings = np.zeros(nb_movies) #We are considering the ratings that have been rated zero. We use numpy function for this
    ratings[id_movies - 1] = id_ratings #We try to access the index of the rating.
    new_data.append(list(ratings)) #we are appending the ratings to the new list
  return new_data 
  #We now apply it to the training and the testing set
training_set = convert(training_set)
test_set = convert(test_set)

Converting the list into the torch tensors 

In [8]:
#Tensors are the multi-dimensional arrays (PyTorch Array)
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Converting the rating into binary format (1 for Liked and 0 for Not Liked)

In [9]:
training_set[training_set == 0] = -1 #All the zero ratings are converted to -1
training_set[training_set == 1] = 0 #Movies given 1 or 2 starts are converted to 0 (NOT LIKED)
training_set[training_set == 2] = 0 #Movies given 1 or 2 starts are converted to 0 (NOT LIKED)
training_set[training_set >= 3] = 1 # Movies that user has given 3+ are converted to 1 (LIKED )
# Same goes for the test set
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

Building the Architecture of the Neural Networks

In [10]:
class RBM(): #Creating the RBM Model
  def __init__(self, nv, nh): #Defining  the parameters of the class
    self.W = torch.randn(nh, nv) #NV- Visible Nodes and NH - Hidden Nodes (MATRIX OF TORCH TENSOR, according to normal distribution)
    self.a = torch.randn(1, nh) #Initialising the visible nodes (MATRIX OF TORCH TENSOR)
    self.b = torch.randn(1, nv) #Initialising the hidden nodes 
  def sample_h(self, x): # To return some samples of the hidden nodes (Using the nodes in self, and x for the visible neurons)
    wx = torch.mm(x, self.W.t()) #mm is used to take product of the 2 tensors 
    activation = wx + self.a.expand_as(wx) #The linear function of the neurons||expand_as is used to add new dimension
    p_h_given_v = torch.sigmoid(activation) #Using the activation function
    return p_h_given_v, torch.bernoulli(p_h_given_v) #Using the bernoulli function || returns the bernoulli sampling
    #Bernoulli neurons work when the probaility falls below the defined value. 
  def sample_v(self, y): # To return some samples of the hidden nodes (Using the nodes in self, and y for the hidden neurons)
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk): #ph0 is the probability of the prediction of hidden nodes and phk is after K sampling
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

TRAINING THE RBM MODEL

In [11]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3438)
epoch: 2 loss: tensor(0.2374)
epoch: 3 loss: tensor(0.2516)
epoch: 4 loss: tensor(0.2482)
epoch: 5 loss: tensor(0.2479)
epoch: 6 loss: tensor(0.2523)
epoch: 7 loss: tensor(0.2473)
epoch: 8 loss: tensor(0.2483)
epoch: 9 loss: tensor(0.2497)
epoch: 10 loss: tensor(0.2474)


Testing the RBM Model

In [12]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2388)
